# Hedonic Feeding Experiment Analysis

`HedonicFeedingExperiment` is a two-well specialisation of `Experiment` designed for experiments that use the choice (two-well) chamber design to study hedonic (reward-based) feeding behaviour.

It adds four features on top of the standard `Experiment` API:

| Method / Attribute | Description |
|---|---|
| `HedonicFeedingExperiment.load()` | Validates that every DFM uses `chamber_size=2` before loading |
| `hedonic_feeding_plot()` | Jitter plot of median bout duration per well, faceted by treatment |
| `filter_flies()` | Remove chambers that fail QC thresholds defined in `flic_config.yaml` |
| `weighted_duration_summary()` | Per-treatment weighted mean and SEM of median bout duration |

`execute_basic_analysis()` runs the standard four steps (QC reports, summary text, feeding summary CSV and plot) and then calls `hedonic_feeding_plot()` and `weighted_duration_summary()`, writing two additional output files.

## Project Directory   
I
t's important to specific the project directory, relative to the location of this notebook. Most often, you will have this notebook in the experiment directory itself and the project directory will be "./".

In [ ]:
# We only need one function from pyflic
from pyflic import HedonicFeedingExperiment
# Point at the example project directory.
project_directory = "./"


## Setting Up the Configuration

Before loading an experiment you need a `flic_config.yaml` in the project directory.  Use the **config editor** to create or modify this file without writing YAML by hand.  The editor lets you define DFMs, assign chambers to treatment groups, set analysis parameters, and specify filter thresholds — all through a GUI.

The config editor can be launched in two ways.

**From the terminal** (recommended — run from inside the project directory so the editor auto-loads any existing config):

```bash
pyflic-config
```

**From this notebook** (opens in a separate window; the notebook remains responsive):

Key settings for a `HedonicFeedingExperiment` to verify in the config:

- **`chamber_size: 2`** — required; the experiment will refuse to load if any DFM has a different value.
- **`well_names`** — optional `A` / `B` labels (e.g. `Sucrose` / `Yeast`) used as axis labels throughout the analysis.
- **`constants`** — the three filter thresholds used by `filter_flies()`: `min_transform_licks_cutoff`, `max_med_duration_cutoff`, and `max_events_cutoff`.

In [ ]:
# Launch the config editor from the notebook.
# Opens in a separate window; run from the project directory so the editor
# auto-loads the existing flic_config.yaml.
import subprocess, os
subprocess.Popen(["pyflic-config"], cwd=str(project_directory))

## Loading the Experiment

`HedonicFeedingExperiment.load()` accepts the same arguments as `Experiment.load()` and reads the same `flic_config.yaml`.  The only difference is that it raises `ValueError` immediately if any DFM has `chamber_size != 2`, so a misconfigured file is caught before any CSV data is loaded.

The `well_names` key in the config (here `A: Sucrose`, `B: Yeast`) is picked up automatically and used as axis labels throughout the analysis.

In [ ]:
exp = HedonicFeedingExperiment.load(project_directory)

## Running the Basic Analysis

`execute_basic_analysis()` runs all standard outputs followed by two hedonic-specific ones:

| Step | Output | Location |
|---|---|---|
| 1 | QC reports — per-DFM integrity check, data-break detection, simultaneous-feeding and bleeding counts, and a raw-data plot | `project_dir/qc/` |
| 2 | Experiment summary text | `project_dir/analysis/summary.txt` |
| 3 | Feeding summary CSV — one row per chamber with all lick, event, and duration metrics | `project_dir/analysis/feeding_summary.csv` |
| 4 | Feeding summary plot — box + jitter panels for every metric | `project_dir/analysis/feeding_summary.png` |
| 5 | Hedonic feeding plot — median bout duration per well, faceted by treatment | `project_dir/analysis/hedonic_feeding_plot.png` |
| 6 | Treatment means CSV — weighted and unweighted duration statistics per treatment | `project_dir/analysis/treatment_means.csv` |

In [ ]:
exp.execute_basic_analysis()

## Reviewing QC

After `execute_basic_analysis()` writes the QC files, use the interactive QC viewer to inspect each DFM before deciding which chambers to exclude.  Launch it from the terminal:

```bash
pyflic-qc /path/to/project_directory
```

The viewer shows raw signal traces, baselined traces, cumulative lick plots, and binned lick histograms for every chamber on every DFM.  Look for:

- **Flat or missing signal** — a well with no detectable baseline activity may indicate a dead fly or a disconnected electrode.
- **Persistently elevated baseline** — suggests a wet or poorly-calibrated well.
- **Data breaks** — long gaps with zero signal, often from wireless transmission interruptions in incubator experiments.
- **Excessive simultaneous feeding** — both wells registering signal at the same sample can inflate or deflate PI estimates.

Record which (DFM, Chamber) pairs to exclude before running `filter_flies()` below.

In [ ]:
# Launch the QC viewer directly from the notebook.
# Opens in a separate window and returns immediately — the notebook stays responsive.
import subprocess
subprocess.Popen(["pyflic-qc", str(project_directory)])

## Filtering Flies

`filter_flies()` removes chambers that fail automated QC thresholds.  The thresholds are read from the `constants` block of `flic_config.yaml`:

| Key | Column tested | Rule |
|---|---|---|
| `min_transform_licks_cutoff` | `LicksA` (power-transformed) | remove if **below** threshold |
| `max_med_duration_cutoff` | `MedDurationA` | remove if **above** threshold |
| `max_events_cutoff` | `EventsB` | remove if **above** threshold |

A chamber is removed if it fails **any** applicable threshold.  Missing keys are silently skipped.

In this experiment the thresholds are:
- `min_transform_licks_cutoff: 0.00001` — removes chambers with effectively zero Sucrose feeding (dead or inactive flies)
- `max_med_duration_cutoff: 13` — removes chambers with implausibly long median bout durations, typically indicating signal artefacts
- `max_events_cutoff: 150` — removes chambers where the Yeast well registers an unrealistically high event count

After filtering the design is updated in place and the feeding-summary cache is cleared, so all subsequent calls use only the surviving chambers.  The removed chambers are stored in `exp.filtered_chambers` and appended to `exp.summary_text()`.

In [ ]:
removed = exp.filter_flies()
removed

## Hedonic Feeding Plot

`hedonic_feeding_plot()` produces a publication-ready jitter plot comparing median bout duration in each well across treatment groups.

- **X-axis** — the two wells, labelled with `well_names` from `flic_config.yaml` (here `Sucrose` and `Yeast`).
- **Y-axis** — median bout duration (seconds) per chamber (`MedDurationA` / `MedDurationB` from the feeding summary).
- **Point size** — scaled by the number of feeding events in that well (`EventsA` or `EventsB`).  Chambers with more events appear as larger dots, giving a visual indication of how much data underlies each point.
- **Colours** — WellA in steel blue, WellB in tomato red.
- **Error bars** — mean ± SEM across chambers within each treatment.
- **Panels** — one facet per treatment group.

By default the plot is saved to `project_dir/analysis/hedonic_feeding_plot.png`.  Pass `save=False` to display without writing a file, as done here to view it inline after filtering.

In [ ]:
p = exp.hedonic_feeding_plot(save=False)
p

## Weighted Duration Summary

`weighted_duration_summary()` computes per-treatment statistics of median bout duration using two averaging strategies.

Weighting by event count down-weights chambers where the fly fed very infrequently, since a median duration based on only a few bouts is less reliable than one based on many.

| Column | Description |
|---|---|
| `MeanDurationA` / `MeanDurationB` | Simple (unweighted) mean of `MedDurationA` / `MedDurationB` across chambers |
| `SemDurationA` / `SemDurationB` | Standard error of the unweighted mean |
| `WeightedMeanDurationA` / `WeightedMeanDurationB` | Weighted mean using `EventsA` / `EventsB` as weights |
| `WeightedSemDurationA` / `WeightedSemDurationB` | Weighted SEM |
| `SampleSizeA` / `SampleSizeB` | Number of chambers with finite weights and durations |

By default the table is saved to `project_dir/analysis/treatment_means.csv`.  Pass `save=False` to return the DataFrame without writing a file.

In [ ]:
means = exp.weighted_duration_summary(save=False)
means